In [1]:
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [4]:
import torch

if torch.cuda.is_available():    
    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [5]:
!pip install transformers

import pandas         as pd

import numpy          as np


import transformers
from transformers                     import  RobertaModel, RobertaTokenizer, AdamW, get_linear_schedule_with_warmup
import torch



from sklearn.metrics                  import classification_report
from sklearn.feature_extraction.text  import TfidfVectorizer

from torch                            import nn, optim
from torch.utils                      import data



RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

if torch.cuda.is_available():
   torch.cuda.manual_seed(RANDOM_SEED)
   torch.cuda.manual_seed_all(RANDOM_SEED) 
   torch.backends.cudnn.deterministic = True  
   torch.backends.cudnn.benchmark = False


CLASS_NAMES = ['support', 'deny', 'query', 'comment']
MAX_LENGTH = 200
BATCH_SIZE = 8
EPOCHS = 6
HIDDEN_UNITS = 128

tokenizer = transformers.RobertaTokenizer.from_pretrained('roberta-large')

     |████████████████████████████████| 4.2 MB 4.1 MB/s 
     |████████████████████████████████| 6.6 MB 89.5 MB/s 
     |████████████████████████████████| 596 kB 75.3 MB/s 
     |████████████████████████████████| 84 kB 4.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

In [6]:
trainDf = pd.read_csv('/content/drive/MyDrive/NLP/csvfiles/trainingData.csv')
trainDf

,replyText,replyTextId,previousText,sourceText,label,previousPlusSrcText,labelValue,pReplyText,pPreviousPlusSrcText,TextSrcInre
0,France: 10 people dead after shooting at HQ of...,5.527836670521672e+17,None,NaN,support,,0.0,France: 10 people dead after shooting at HQ of...,NaN,France: 10 people dead after shooting at HQ of...
1,BREAKING: 10 reportedly shot dead at Paris HQ ...,5.5278537516149965e+17,None,NaN,support,,0.0,BREAKING: 10 reportedly shot dead at Paris HQ ...,NaN,BREAKING: 10 reportedly shot dead at Paris HQ ...
2,BREAKING: At least 10 killed in shooting at Fr...,5.527911962472694e+17,None,NaN,support,,0.0,BREAKING: At least 10 killed in shooting at Fr...,NaN,BREAKING: At least 10 killed in shooting at Fr...
3,Eleven dead in shooting at Paris offices of sa...,5.527915788936192e+17,None,NaN,support,,0.0,Eleven dead in shooting at Paris offices of sa...,NaN,Eleven dead in shooting at Paris offices of sa...
4,BREAKING Charlie Hebdo latest: 11 dead 10 woun...,5.527925441329971e+17,None,NaN,support,,0.0,BREAKING Charlie Hebdo latest: 11 dead 10 woun...,NaN,BREAKING Charlie Hebdo latest: 11 dead 10 woun...
...,...,...,...,...,...,...,...,...,...,...
5212,"&gt; That isn't disputed, it's that he came to...",e2bmzf2,e2bmcct,8yktu5,comment,"That isn't disputed, it's that he came to Scot...",3.0,"&gt; That isn't disputed, it's that he came to...","That isn't disputed, it's that he came to Scot...","&gt; That isn't disputed, it's that he came to..."
5213,In the press conference earlier today he said ...,e2bos7p,e2bo412,8yktu5,deny,There is no point to any of this. Jon ...,1.0,In the press conference earlier today he said ...,There is no point to any of this. Jon ...,In the press conference earlier today he said ...
5214,[deleted],e2btp0f,e2bta92,8yktu5,comment,Lol it doesn't though Jon Sopel: Bizarre. @rea...,3.0,[deleted],Lol it doesn't though Jon Sopel: Bizarre. $MEN...,[deleted] Lol it doesn't though Jon Sopel: Biz...
5215,[deleted],e2bxvw0,e2bxjzw,8yktu5,comment,"It's just another of the many ""factual errors""...",3.0,[deleted],"It's just another of the many ""factual errors""...","[deleted] It's just another of the many ""factu..."


In [7]:
trainDf=trainDf.drop(trainDf.index[841])

In [8]:
trainDf['label'].isna().sum()

0

In [9]:
devDf = pd.read_csv('/content/drive/MyDrive/NLP/csvfiles/devData.csv')
devDf

,replyText,replyTextId,previousText,sourceText,label,previousPlusSrcText,labelValue,pReplyText,pPreviousPlusSrcText,TextSrcInre
0,Appalled by the attack on Charlie Hebdo in Par...,552788945017516032,None,NaN,support,,0,Appalled by the attack on Charlie Hebdo in Par...,NaN,Appalled by the attack on Charlie Hebdo in Par...
1,Reports of fatality and injuries following sho...,553480082996879360,None,NaN,support,,0,Reports of fatality and injuries following sho...,NaN,Reports of fatality and injuries following sho...
2,#BREAKING Paris hostage-taker 'knows' one Char...,553553288625672192,None,NaN,support,,0,#BREAKING Paris hostage-taker 'knows' one Char...,NaN,#BREAKING Paris hostage-taker 'knows' one Char...
3,BREAKING: Police order all shops closed in fam...,553561170637238272,None,NaN,support,,0,BREAKING: Police order all shops closed in fam...,NaN,BREAKING: Police order all shops closed in fam...
4,Every 28 hours a black male is killed in the U...,498293668655423488,None,NaN,support,,0,Every 28 hours a black male is killed in the U...,NaN,Every 28 hours a black male is killed in the U...
...,...,...,...,...,...,...,...,...,...,...
1480,Remember the fluorides in our water replaces w...,e3bk3q8,934q6t,934q6t,comment,Iodine increases IQ and is an essential part o...,3,Remember the fluorides in our water replaces w...,Iodine increases IQ and is an essential part o...,Remember the fluorides in our water replaces w...
1481,"We evolved as omnivores, but I do agree with y...",e3ay5vh,e3ak72f,934q6t,comment,Oh they put iodine in salt here in Latin Ameri...,3,"We evolved as omnivores, but I do agree with y...",Oh they put iodine in salt here in Latin Ameri...,"We evolved as omnivores, but I do agree with y..."
1482,&gt; 7 drops in about a double shot of cold wa...,e3c5joo,e3bqum8,934q6t,comment,I do about 7 drops in about a double shot of c...,3,&gt; 7 drops in about a double shot of cold wa...,I do about 7 drops in about a double shot of c...,&gt; 7 drops in about a double shot of cold wa...
1483,The book The Iodine Crisis is really good and ...,e3bq78h,934q6t,934q6t,support,Iodine increases IQ and is an essential part o...,0,The book The Iodine Crisis is really good and ...,Iodine increases IQ and is an essential part o...,The book The Iodine Crisis is really good and ...


In [10]:
testDf = pd.read_csv('/content/drive/MyDrive/NLP/csvfiles/testData.csv')
testDf

,replyText,replyTextId,previousText,sourceText,label,previousPlusSrcText,labelValue,pReplyText,pPreviousPlusSrcText,TextSrcInre
0,#Breaking: Pentagon releases video of the “mot...,852844529778601985,None,NaN,support,,0,#Breaking: Pentagon releases video of the “mot...,NaN,#Breaking: Pentagon releases video of the “mot...
1,Rep. Sheila Jackson Lee has no shame. I still ...,443938194715713536,None,NaN,support,,0,Rep. Sheila Jackson Lee has no shame. I still ...,NaN,Rep. Sheila Jackson Lee has no shame. I still ...
2,"ICYMI: ""Rep Sheila Jackson Lee (D-Tx) Wants Hu...",774165935041093633,None,NaN,support,,0,"ICYMI: ""Rep Sheila Jackson Lee (D-Tx) Wants Hu...",NaN,"ICYMI: ""Rep Sheila Jackson Lee (D-Tx) Wants Hu..."
3,"In 2003, Houston U.S. Rep. Sheila Jackson Lee ...",909084965035487232,None,NaN,support,,0,"In 2003, Houston U.S. Rep. Sheila Jackson Lee ...",NaN,"In 2003, Houston U.S. Rep. Sheila Jackson Lee ..."
4,Noah’s Ark Theme Park Destroyed in a Flood? ht...,897502435593265152,None,NaN,query,,2,Noah’s Ark Theme Park Destroyed in a Flood? $URL$,NaN,Noah’s Ark Theme Park Destroyed in a Flood? $U...
...,...,...,...,...,...,...,...,...,...,...
1822,Sometimes you can't win an argument. :-)\n\nAm...,c5nsrhe,xn2bn,xn2bn,comment,"I've been searching, and can't find a single c...",3,Sometimes you can't win an argument. :-)\n\nAm...,"I've been searching, and can't find a single c...",Sometimes you can't win an argument. :-)\n\nAm...
1823,"I'm not a troll.. I figured it was bullshit, b...",c5nsqr2,c5nspru,xn2bn,comment,Just in case this isn't just a troll\n\n- Obam...,3,"I'm not a troll.. I figured it was bullshit, b...",Just in case this isn't just a troll\n\n- Obam...,"I'm not a troll.. I figured it was bullshit, b..."
1824,Just in case this isn't just a troll\n\n- Obam...,c5nspru,xn2bn,xn2bn,comment,"I've been searching, and can't find a single c...",3,Just in case this isn't just a troll\n\n- Obam...,"I've been searching, and can't find a single c...",Just in case this isn't just a troll\n\n- Obam...
1825,Right-wing republicans? Aren't a lot of them h...,c5nsn66,xn2bn,xn2bn,comment,"I've been searching, and can't find a single c...",3,Right-wing republicans? Aren't a lot of them h...,"I've been searching, and can't find a single c...",Right-wing republicans? Aren't a lot of them h...


In [11]:
class StanceDataset(data.Dataset):

  def __init__(self, firstSeq, secondSeq, labelValue,  tokenizer, max_len):
    self.firstSeq    = firstSeq      
    self.secondSeq   = secondSeq     
    self.labelValue  = labelValue    
    self.tokenizer   = tokenizer    
    self.max_len     = max_len       

  def __len__(self):
    return len(self.labelValue)

  def __getitem__(self, item):
    firstSeq    = str(self.firstSeq[item])
    secondSeq   = str(self.secondSeq[item])
 
  
    encoding = tokenizer.encode_plus(
        firstSeq,
        secondSeq,
        max_length = self.max_len,
        add_special_tokens= True,
        truncation = True,
        pad_to_max_length = True,
        return_attention_mask = True,
        return_tensors = 'pt'
    )

    return {
        'firstSeq' : firstSeq,
        'secondSeq' : secondSeq,
        'input_ids': encoding['input_ids'].flatten(),
        'attention_mask': encoding['attention_mask'].flatten(),
        'labelValue'  : torch.tensor(self.labelValue[item], dtype=torch.long)
    }

In [12]:

def createDataLoader(dataframe, tokenizer, max_len, batch_size):
  ds = StanceDataset(
      firstSeq    = dataframe.pReplyText.to_numpy(),
      secondSeq   = dataframe.pPreviousPlusSrcText.to_numpy(),
      labelValue  = dataframe.labelValue.to_numpy(),
      tokenizer   = tokenizer,
      max_len     = max_len
  )

  return data.DataLoader(
      ds,
      batch_size  = batch_size,
      shuffle     = True,
      num_workers = 4
  )


In [13]:

trainDataLoader        = createDataLoader(trainDf, tokenizer, MAX_LENGTH, BATCH_SIZE)

developmentDataLoader  = createDataLoader(devDf, tokenizer, MAX_LENGTH, BATCH_SIZE)



In [15]:
class StanceClassifier(nn.Module):

  def __init__(self,  n_classes):
    super(StanceClassifier, self).__init__()
    self.robertaModel              = RobertaModel.from_pretrained('roberta-large')                                  
    self.drop                      = nn.Dropout(p = 0.3)
    self.output                    = nn.Linear(self.robertaModel.config.hidden_size, n_classes)
    self.softmax                   = nn.Softmax(dim = 1)

  def forward(self, input_ids, attention_mask):
    
    roberta_output     = self.robertaModel(
        input_ids      = input_ids,               
        attention_mask = attention_mask )         
    pooled_output = roberta_output[1]          
    output        = self.drop(pooled_output)
    output        = self.output(output)

    return self.softmax(output)

In [16]:
model = StanceClassifier(len(CLASS_NAMES))
model = model.to(device)

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
optimizer = AdamW(model.parameters(), 
                  lr = 2e-6, 
                  correct_bias= False)

totalSteps = len(trainDataLoader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=0,
            num_training_steps = totalSteps
)



classWeights = torch.FloatTensor([4.0, 42.0, 4.0, 1.0])
lossFunction = nn.CrossEntropyLoss(weight = classWeights).to(device)

In [18]:
def train_epoch(
  model,
  dataLoader,
  lossFunction,
  optimizer,
  device,
  scheduler,
  n_examples
):

  model = model.train()
  losses = []
  correctPredictions = 0

  for d in dataLoader:
    
    input_ids              = d["input_ids"].to(device)                           
    attention_mask         = d["attention_mask"].to(device)                     
    labelValues            = d["labelValue"].to(device)                          
                                      

    outputs = model(
      input_ids             = input_ids,
      attention_mask        = attention_mask,
    )

   
    _, predictionIndices = torch.max(outputs, dim=1)
    loss = lossFunction(outputs, labelValues)


    correctPredictions += torch.sum(predictionIndices == labelValues)
    losses.append(loss.item())
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

  return np.mean(losses), correctPredictions.double() / n_examples

In [19]:
def eval_model(
    model, 
    dataLoader, 
    lossFunction,
    device,
    n_examples
    ):
  
  model = model.eval()
  losses = []
  correctPredictions = 0

  with torch.no_grad():
    for d in dataLoader:

      input_ids              = d["input_ids"].to(device)                          
      attention_mask         = d["attention_mask"].to(device)                     
      labelValues            = d["labelValue"].to(device)                        
   
      outputs = model(
        input_ids             = input_ids,
        attention_mask        = attention_mask,
  
      )

      _, predictionIndices = torch.max(outputs, dim=1)
      loss = lossFunction(outputs, labelValues)

      correctPredictions += torch.sum(predictionIndices == labelValues)
      losses.append(loss.item())

  return np.mean(losses), correctPredictions.double() / n_examples

In [20]:
for epoch in range(EPOCHS):
  print(f'Epoch {epoch + 1}')
  trainLoss, trainAccuracy = train_epoch(
    model,
    trainDataLoader,
    lossFunction,
    optimizer,
    device,
    scheduler,
    len(trainDf)
  )
  
  print(f'Training loss {trainLoss} Training accuracy {trainAccuracy}')

  devLoss, devAccuracy = eval_model(
    model,
    developmentDataLoader,
    lossFunction,
    device,
    len(devDf)
  )

  print(f'Development loss {devLoss} Development accuracy {devAccuracy}')
  print()
  
  print()

Epoch 1


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Training loss 1.2993561984296957 Training accuracy 0.6284509202453988


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Development loss 1.2227881619686722 Development accuracy 0.7925925925925926


Epoch 2


...

Development loss 1.0498677442471187 Development accuracy 0.8296296296296297


Epoch 6


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Training loss 1.1117831737808654 Training accuracy 0.7870015337423313


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Development loss 1.0492377696178292 Development accuracy 0.8249158249158249




In [21]:
def get_predictions(model, data_loader):

  model = model.eval()
  review_texta = []
  review_textb = []
  predictions = []
  prediction_probs = []
  real_values = []

  with torch.no_grad():
    for d in data_loader:

      textas                 = d["firstSeq"]
      textbs                 = d["secondSeq"]
      input_ids              = d["input_ids"].to(device)
      attention_mask         = d["attention_mask"].to(device)
      labels                 = d["labelValue"].to(device)
    
   
      outputs = model(
        input_ids             = input_ids,
        attention_mask        = attention_mask,
      )

      _, preds = torch.max(outputs, dim=1)    

      review_texta.extend(textas)
      review_textb.extend(textbs)
      predictions.extend(preds)
      prediction_probs.extend(outputs)
      real_values.extend(labels)

  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  
  return review_texta, review_textb, predictions, prediction_probs, real_values

In [22]:
firstSeq_dev, secondSeq_dev, yHat_dev, predProbs_dev, yTest_dev = get_predictions(
  model,
  developmentDataLoader
)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [23]:
print(classification_report(yTest_dev, yHat_dev, target_names= CLASS_NAMES))

              precision    recall  f1-score   support

     support       0.45      0.30      0.36       102
        deny       0.37      0.43      0.40        82
       query       0.73      0.72      0.73       120
     comment       0.89      0.91      0.90      1181

    accuracy                           0.82      1485
   macro avg       0.61      0.59      0.60      1485
weighted avg       0.82      0.82      0.82      1485



In [24]:
yTest_dev

tensor([3, 3, 1,  ..., 3, 3, 3])

In [25]:
yHat_dev

tensor([3, 3, 1,  ..., 3, 3, 3])

In [27]:
for i in range(len(yTest_dev)):
  if yTest_dev[i] == yHat_dev[i]:
    continue
  else:
    print("actual"+str(yTest_dev[i]))
    print("predicted"+str(yHat_dev[i]))

actualtensor(2)
predictedtensor(3)
actualtensor(3)
predictedtensor(2)
actualtensor(3)
predictedtensor(1)
actualtensor(3)
predictedtensor(1)
actualtensor(2)
predictedtensor(3)
actualtensor(0)
predictedtensor(3)
actualtensor(3)
predictedtensor(1)
actualtensor(0)
predictedtensor(3)
actualtensor(3)
predictedtensor(1)
actualtensor(3)
predictedtensor(1)
actualtensor(0)
predictedtensor(2)
actualtensor(1)
predictedtensor(3)
actualtensor(3)
predictedtensor(2)
actualtensor(0)
predictedtensor(3)
actualtensor(3)
predictedtensor(1)
actualtensor(3)
predictedtensor(0)
actualtensor(3)
predictedtensor(2)
actualtensor(0)
predictedtensor(3)
actualtensor(2)
predictedtensor(3)
actualtensor(3)
predictedtensor(1)
actualtensor(0)
predictedtensor(3)
actualtensor(3)
predictedtensor(1)
actualtensor(2)
predictedtensor(3)
actualtensor(0)
predictedtensor(1)
actualtensor(0)
predictedtensor(3)
actualtensor(2)
predictedtensor(3)
actualtensor(0)
predictedtensor(3)
actualtensor(1)
predictedtensor(3)
actualtensor(3)
pred